# CNN-LSTM Model

## Setup

In [1]:
# !pip install -q --upgrade pip
# !pip install -q tensorflow tensorflow-metal scikit-learn ipywidgets
# !pip install -q scipy

In [3]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import tensorflow as tf
import datetime
import pandas as pd

In [4]:
patch_size_pixels = 64
model_input_width = 6  # number of images before the event
buffer_size = 100
# num_classes = 6
batch_size = 8


datasets_folder = os.path.join("E:\\Workspace\\Thesis_dataset", "illinois_dataset")
# dataset_folder_name = "Quercus_germany_200_stations"

In [5]:
dataset_folder_path = datasets_folder
os.makedirs(dataset_folder_path, exist_ok=True)

record_id_column_name = 'record_id'
label_column_name = "ordered_phase_id"

train_timeseries_file_path = os.path.join(dataset_folder_path, "test_data.npz")
print(train_timeseries_file_path)
train_label_file_path = os.path.join(dataset_folder_path, "test.csv")
print(train_label_file_path)
print("-" * 60)
# val_timeseries_file_path = os.path.join(dataset_folder_path, "val_data.npz")
# print(val_timeseries_file_path)
# val_label_file_path = os.path.join(dataset_folder_path, "val.csv")
# print(val_label_file_path)
# print("-" * 60)
# test_timeseries_file_path = os.path.join(dataset_folder_path, "test_data.npz")
# print(test_timeseries_file_path)
# test_label_file_path = os.path.join(dataset_folder_path, "test.csv")
# print(test_label_file_path)

E:\Workspace\Thesis_dataset\illinois_dataset\test_data.npz
E:\Workspace\Thesis_dataset\illinois_dataset\test.csv
------------------------------------------------------------


In [6]:
npz_file = np.load(train_timeseries_file_path, mmap_mode="r")
first_array_key = next(iter(npz_file))
timeseries_shape = npz_file[first_array_key].shape
timeseries_shape

(6, 64, 64, 15)

## Create the datasets

### Create the train, validation, and test dataset

In [ ]:
label_csv_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\train.csv"
test_label_file_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\test.csv"
record_id_column_name = "id"
label_column_name = "val_yield"
timeseries_npz_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\train_data.npz"
test_timeseries_file_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\train_data.npz"

In [203]:
def data_generator(
    label_csv_path, record_id_column_name, label_column_name, timeseries_npz_path
):

    # Check if the inputs are a byte string and decode it to a regular string if necessary
    if isinstance(label_csv_path, bytes):
        label_csv_path = label_csv_path.decode("utf-8")
    if isinstance(timeseries_npz_path, bytes):
        timeseries_npz_path = timeseries_npz_path.decode("utf-8")
    if isinstance(record_id_column_name, bytes):
        record_id_column_name = record_id_column_name.decode("utf-8")
    if isinstance(label_column_name, bytes):
        label_column_name = label_column_name.decode("utf-8")

    # Load the labels CSV
    labels_df = pd.read_csv(label_csv_path)

    # Load the npz file
    with np.load(timeseries_npz_path, allow_pickle=True) as npz_file:
        for _, row in labels_df.iterrows():
            record_id = str(
                row[record_id_column_name]
            )  # Ensure record_id is treated as a string
            target = row[label_column_name]
            if record_id in npz_file.files:
                # Extract the time series data using record_id
                time_series = npz_file[record_id]
                # print(record_id, target)
                yield time_series, target


# Determine the output types
output_types = (tf.float32, tf.int64)

# Determine the output shapes
output_shapes = (timeseries_shape, ())  #  ((6, 11, 64, 64), ())

# Create train dataset
train_dataset = tf.data.Dataset.from_generator(
    data_generator,  # Generator function
    args=(
        train_label_file_path,
        record_id_column_name,
        label_column_name,
        train_timeseries_file_path,
    ),  # Arguments to pass to the generator
    output_types=output_types,
    output_shapes=output_shapes,
)
train_dataset = (
    train_dataset.shuffle(buffer_size=buffer_size)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# # Create validation dataset
# val_dataset = tf.data.Dataset.from_generator(
#     data_generator,  # Generator function
#     args=(
#         val_label_file_path,
#         record_id_column_name,
#         label_column_name,
#         val_timeseries_file_path,
#     ),  # Arguments to pass to the generator
#     output_types=output_types,
#     output_shapes=output_shapes,
# )
# val_dataset = (
#     # val_dataset.shuffle(buffer_size=buffer_size)
#     val_dataset.batch(batch_size)
#     .prefetch(tf.data.AUTOTUNE)
# )

# Create test dataset
test_dataset = tf.data.Dataset.from_generator(
    data_generator,  # Generator function
    args=(
        test_label_file_path,
        record_id_column_name,
        label_column_name,
        test_timeseries_file_path,
    ),  # Arguments to pass to the generator
    output_types=output_types,
    output_shapes=output_shapes,
)
test_dataset = test_dataset.batch(batch_size)

Print the shape of the datasets

In [204]:
# Function to print the shapes of the dataset
def print_dataset_shapes(dataset, dataset_name="Dataset"):
    print(f"--- {dataset_name} ---")
    for data, label in dataset.take(1):  # Take a single batch from the dataset
        print(f"Shape of data: {data.shape}")
        print(f"Shape of label: {label.shape}")

# Print shapes for train, validation, and test datasets
print_dataset_shapes(train_dataset, "Train Dataset")
# print_dataset_shapes(val_dataset, "Validation Dataset")
print_dataset_shapes(test_dataset, "Test Dataset")



--- Train Dataset ---
Shape of data: (8, 6, 64, 64, 15)
Shape of label: (8,)
--- Test Dataset ---
Shape of data: (8, 6, 64, 64, 15)
Shape of label: (8,)


In [205]:
# Apply interpolation for band 13 on the train dataset
train_dataset = apply_interpolation_to_band(train_dataset, band_num=13)
val_dataset = apply_interpolation_to_band(val_dataset, band_num=13)
test_dataset = apply_interpolation_to_band(test_dataset, band_num=13)

# Apply interpolation for band 14 on the train dataset
train_dataset = apply_interpolation_to_band(train_dataset, band_num=14)
val_dataset = apply_interpolation_to_band(val_dataset, band_num=14)
test_dataset = apply_interpolation_to_band(test_dataset, band_num=14)

NameError: name 'apply_interpolation_to_band' is not defined

### Normalization

In [206]:
def compute_normalization_stats(dataset):
    """
    Compute the mean and standard deviation of the dataset.
    
    Args:
    dataset: A TensorFlow Dataset from which the statistics should be computed.

    Returns:
    mean: Mean of the dataset.
    stddev: Standard deviation of the dataset.
    """
    # Initialize variables to compute mean and variance
    total_sum = 0
    total_squared_sum = 0
    total_count = 0

    # Iterate through the dataset to compute the sum and squared sum
    for data, _ in dataset:
        total_sum += tf.reduce_sum(data, axis=0)
        total_squared_sum += tf.reduce_sum(tf.square(data), axis=0)
        total_count += tf.cast(tf.shape(data)[0], tf.float32)

    # Compute the mean and variance
    mean = total_sum / total_count
    variance = (total_squared_sum / total_count) - tf.square(mean)
    stddev = tf.sqrt(variance)
    
    return mean, stddev

def normalize_dataset(dataset, mean, stddev):
    """
    Normalize the dataset using mean and standard deviation, replacing NaN values with the mean.

    Args:
    dataset: A TensorFlow Dataset that needs to be normalized.
    mean: The mean values for each feature.
    stddev: The standard deviation for each feature.

    Returns:
    A normalized TensorFlow Dataset.
    """
    # Function to replace NaN values with the mean and normalize
    def replace_nan_and_normalize(data, target):
        # Replace NaN values with the mean
        data_no_nan = tf.where(tf.math.is_nan(data), mean, data)
        # Apply normalization using broadcasting
        normalized_data = (data_no_nan - mean) / stddev
        return normalized_data, target

    # Apply the replace and normalization function
    return dataset.map(replace_nan_and_normalize)


In [207]:
# Assuming train_dataset is a tf.data.Dataset object
mean, stddev = compute_normalization_stats(train_dataset)

# Normalize the train dataset
train_dataset = normalize_dataset(train_dataset, mean, stddev)

# Normalize the validation dataset
# ''val_dataset = normalize_dataset(val_dataset, mean, stddev)
''
# Normalize the test dataset
test_dataset = normalize_dataset(test_dataset, mean, stddev)


Print the first record

In [208]:
# Function to print the first record in the dataset
def print_first_record(dataset, dataset_name="Dataset"):
    print(f"--- First Record from {dataset_name} ---")
    for data, label in dataset.take(1):  # Take a single batch from the dataset
        print(f"Data: {data[0]}")
        print(f"Label: {label[0]}")

# Print the first record for the train dataset
print_first_record(train_dataset, "Train Dataset")


--- First Record from Train Dataset ---
Data: [[[[-0.83323246 -0.4531153  -0.4134176  ...         nan  0.9348566
     0.81256604]
   [-0.83323246 -0.46487066 -0.4436577  ...         nan  0.9350435
     0.8125327 ]
   [-0.780351   -0.47659656 -0.46685538 ...         nan  0.93527263
     0.8124918 ]
   ...
   [-0.4953167  -0.6231799  -0.6443652  ...         nan  0.9338001
     0.8051857 ]
   [-0.54009897 -0.6119555  -0.6391573  ...         nan  0.9333845
     0.8051065 ]
   [-0.54009897 -0.6020085  -0.631915   ...         nan  0.93308604
     0.80504197]]

  [[-0.83323246 -0.45687667 -0.42399523 ...         nan  0.93467456
     0.8126163 ]
   [-0.83323246 -0.4635476  -0.4428054  ...         nan  0.93487054
     0.8125837 ]
   [-0.780351   -0.4704841  -0.45715842 ...         nan  0.93511486
     0.81254345]
   ...
   [-0.4953167  -0.6285186  -0.64732325 ...         nan  0.93441504
     0.8052496 ]
   [-0.54009897 -0.6184831  -0.64215744 ...         nan  0.9339787
     0.80517006]
   [-0.5

## Base training and test functions

In [209]:
# Clear any logs from previous runs
# !rm -rf ./logs/

In [210]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [211]:
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

def train_model(model, num_epochs=10):
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Custom learning rate scheduler
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


    # Define the path where the best model will be saved
    best_model_filepath = f"{log_dir}/best_model.h5"
    
    # ModelCheckpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(
        best_model_filepath,
        monitor='val_accuracy',  # Change to monitor validation accuracy
        save_best_only=True,
        mode='max',  # Change to 'max' for accuracy, since we want the highest value
        verbose=1
    )

    callbacks_list = [
        tensorboard_callback,
        # LearningRateScheduler(custom_lr_scheduler, verbose=1),
        checkpoint_callback  # Add ModelCheckpoint to callbacks
    ]
    # Train the model
    history = model.fit(
        train_dataset,
        epochs=num_epochs,
        callbacks=callbacks_list,
        # validation_data=val_dataset,
    )

    # Load the best weights from the saved model
    model.load_weights(best_model_filepath)

    # Evaluate the best model on the training dataset
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    
    # Evaluate the best model on the validation dataset
    val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)

    # Print training and validation accuracy
    print(f"Best model training accuracy: {train_accuracy:.4f}")
    print(f"Best model validation accuracy: {val_accuracy:.4f}")


    # Return the model with the best weights
    return model


## CNN-LSTM

### Define the model

In [212]:
# Clean up the column names to remove any hidden spaces or non-printing characters
labels_df.columns = labels_df.columns.str.strip()

# Now, you should be able to access 'corn_y_bu_per_m2' without any issues
print(labels_df.columns)


Index(['Unnamed: 0', 'id', 'lon', 'lat', 'soy_y_bu_per_m2', 'corn_y_bu_per_m2',
       'soy_y_bu_per_patch', 'corn_y_bu_per_patch'],
      dtype='object')


In [10]:
from tensorflow.keras import layers, models
from tensorflow.keras.losses import MeanAbsoluteError

def create_cnn_lstm_model(input_shape):
    # Model definition
    model = models.Sequential()
    
    # TimeDistributed wrapper applies a layer to every temporal slice of an input.
    model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding='same'), input_shape=input_shape))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    
    model.add(layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

    model.add(layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu', padding='same')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    
    # Flatten the spatial dimensions, but keep the temporal dimension
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    # LSTM layer to process the temporal sequence
    model.add(layers.LSTM(256, return_sequences=False))
    
    # Dense layers for classification
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1))
    
    return model


### Construct the model 

In [11]:

model_cnn_lstm = create_cnn_lstm_model(timeseries_shape)

model_cnn_lstm.compile(
    optimizer="adam", loss=MeanAbsoluteError(), metrics=[MeanAbsoluteError()]
)

model_cnn_lstm.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_7              │ (None, 6, 64, 64, 64)  │         8,704 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 6, 32, 32, 64)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 6, 32, 32, 32)  │        18,464 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 6, 16, 16, 32)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 6, 16, 16, 16)  │         4,624 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_12             │ (None, 6, 8, 8, 16)    │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_13             │ (None, 6, 1024)        │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,606,705 (6.13 MB)

 Trainable params: 1,606,705 (6.13 MB)

 Non-trainable params: 0 (0.00 B)

### Train

In [ ]:
model_cnn_lstm = train_model(model_cnn_lstm, 10)

In [169]:
# Check the number of batches in the test dataset
dataset_length = sum(1 for _ in test_dataset)
print(f"Test dataset contains {dataset_length} batches.")


Test dataset contains 20 batches.


In [ ]:
# If the dataset only contains features and no labels for test data
# Predict on the test dataset and do not evaluate accuracy
predictions = model_cnn_lstm.predict(test_dataset)

# If predictions are in a NumPy array, convert it to a Pandas DataFrame
predictions_df = pd.DataFrame(predictions)

# Save the predictions to an Excel file (make sure to use the correct file path)
predictions_df.to_excel('predictions.xlsx', index=False, header=False)  # Adjust 'header' as needed


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


c:\Users\Surya Naganathan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model_cnn_lstm.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
predictions = model_cnn_lstm.predict(test_dataset)
print(predictions)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
[0.33050215 0.13389957 0.13389957 0.13389957 0.13389957 0.13389957]


c:\Users\Surya Naganathan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model_cnn_lstm.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")